<br>
<br>

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Modelos seq2seq 2**

Cuando la entrada de cada paso del decodificador proviene de la salida del paso anterior, estamos hablando de un modelo Seq2Seq con realimentación (feedback). Esto es especialmente común en tareas como la generación de texto.

1. **Inicio de la Secuencia**: Se inicia la generación con un token especial, como `<SOS>` (Start of Sequence).

2. **Generación Paso a Paso**:
   - En el primer paso, el decodificador recibe el `<SOS>` y el estado del codificador como entrada.
   - El decodificador procesa esta entrada y genera una salida para este paso.
   - La salida generada se convierte en la entrada para el siguiente paso, junto con el estado actualizado del decodificador.
   - Este proceso se repite hasta que se genera un token especial de fin de secuencia (`<EOS>`, End of Sequence) o hasta alcanzar un límite máximo de longitud.

3. **Ventajas y Desventajas**:
   - **Ventajas**: Esta forma de generar secuencias puede ayudar a mantener la coherencia en las secuencias generadas, ya que cada nueva palabra o token tiene en cuenta lo que ya se ha generado.
   - **Desventajas**: Puede ser más lento, ya que cada paso depende del anterior, y errores en un paso pueden propagarse y afectar los pasos siguientes.


Imagina que tienes un modelo seq2seq entrenado para traducir inglés a español. Para la frase "How are you?", el proceso sería algo así:

1. El codificador procesa "How are you?" y genera un vector latente.
2. El decodificador recibe el vector latente y el token `<SOS>`.
3. El decodificador genera "¿Cómo", actualiza su estado y toma "¿Cómo" como entrada para el siguiente paso.
4. El decodificador genera "estás", actualiza su estado y toma "estás" como entrada para el siguiente paso.
5. Y así sucesivamente, hasta generar `<EOS>` para indicar el final de la secuencia.

Este modelo de generación permite que el decodificador tenga en cuenta no solo el contexto proporcionado por el codificador, sino también la estructura de la secuencia que está generando, paso a paso.


<p align="center">
<img src="imgs/seq2seq-feedback.svg" width="70%">
</p>


### **Teacher Forcing**

**Teacher forcing** es una técnica utilizada en el entrenamiento de modelos seq2seq en la que, en un porcentaje de las veces, se utiliza la salida real (etiqueta) de un paso de tiempo como entrada para el siguiente paso, en lugar de la salida predicha por el modelo. Esta técnica puede ayudar a acelerar la convergencia y mejorar el rendimiento del modelo, especialmente en las etapas iniciales del entrenamiento.

#### **¿Cómo funciona?**

1. **Durante el Entrenamiento**: En cada paso de tiempo y con una cierta probabilidad de que suceda, en lugar de pasar la predicción del modelo del paso anterior como entrada al siguiente paso, se pasa la palabra real de la secuencia objetivo. Esto proporciona al modelo información directa y clara sobre cómo debería haber respondido en el paso anterior, independientemente de si la predicción fue correcta o no.

2. **Durante la Evaluación/Predicción**: El modelo debe generar secuencias por sí mismo, utilizando sus propias predicciones del paso anterior para el siguiente paso. Durante esta fase, no se utiliza "teacher forcing".

#### **Ventajas de Teacher Forcing:**

1. **Aprendizaje más Rápido**: Al proporcionar al modelo la respuesta correcta en cada paso, se reduce la propagación de errores a través de la secuencia, lo que puede llevar a un aprendizaje más rápido.

2. **Mejor Rendimiento**: Puede resultar en un mejor rendimiento del modelo, especialmente en las primeras etapas del entrenamiento.

### **Implementación traductor inglés a español**

#### **Dataset Europarl**

El conjunto de datos Europarl contiene las transcripciones de los procedimientos del Parlamento Europeo, proporcionando una valiosa fuente de textos paralelos en 21 idiomas europeos. Las oraciones están alineadas entre los idiomas, lo que lo hace especialmente útil para tareas de traducción automática. 

Descargamos el dataset Europarl para español-inglés. Una vez descargado tendremos dos ficheros de texto, uno para cada idioma con las frases alineadas. El código siguiente muestra las primeras frases de cada fichero.

In [2]:
def leer_europarl(archivo_ingles, archivo_espanol):
    with open(archivo_ingles, 'r', encoding='utf-8') as f_ingles, open(archivo_espanol, 'r', encoding='utf-8') as f_espanol:
        for oracion_ingles, oracion_espanol in zip(f_ingles, f_espanol):
            yield oracion_ingles.strip(), oracion_espanol.strip()


archivo_ingles = 'data/europarl/europarl-v7.es-en.en'
archivo_espanol = 'data/europarl/europarl-v7.es-en.es'

# Leer el conjunto de datos
for i, (ingles, espanol) in enumerate(leer_europarl(archivo_ingles, archivo_espanol)):
    print('Inglés:', ingles)
    print('Español:', espanol)
    print('---')
    if i == 15:
        break

Inglés: Resumption of the session
Español: Reanudación del período de sesiones
---
Inglés: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Español: Declaro reanudado el período de sesiones del Parlamento Europeo, interrumpido el viernes 17 de diciembre pasado, y reitero a Sus Señorías mi deseo de que hayan tenido unas buenas vacaciones.
---
Inglés: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
Español: Como todos han podido comprobar, el gran "efecto del año 2000" no se ha producido. En cambio, los ciudadanos de varios de nuestros países han sido víctimas de catástrofes naturales verdaderamente terribles.
---
Inglés: You have requested a debate on this subject in the course of the next

In [26]:
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import torchtext
import torch

class EuroparlDataset(Dataset):
    def __init__(self):
        self.ingles = []
        self.espanol = []
        self.tokenizer_es = get_tokenizer("spacy", language="es_core_news_md")
        self.tokenizer_en = get_tokenizer("spacy", language="en_core_web_md")
        self.vocab_es = torchtext.vocab.FastText(language='es')
        self.vocab_en = torchtext.vocab.FastText(language='en')

        self.vocab_en = self.add_sos_eos(self.vocab_en)
        self.vocab_es = self.add_sos_eos(self.vocab_es)
        
        self.archivo_ingles = 'data/europarl/europarl-v7.es-en.en'
        self.archivo_espanol = 'data/europarl/europarl-v7.es-en.es'

        # Leer el conjunto de datos
        for ingles, espanol in self.leer_europarl():
            self.ingles.append(ingles)
            self.espanol.append(espanol)


    def add_sos_eos(self, vocabulary):
        words = vocabulary.itos
        vocab = vocabulary.stoi
        embedding_matrix = vocabulary.vectors

        # Tokens especiales
        sos_token = '<sos>'
        eos_token = '<eos>'

        # Inicializamos los vectores para los tokens especiales, por ejemplo, con ceros
        sos_vector = torch.zeros(1, embedding_matrix.shape[1])
        eos_vector = torch.zeros(1, embedding_matrix.shape[1])

        # Añade los vectores al final de la matriz de embeddings
        embedding_matrix = torch.cat((embedding_matrix, sos_vector, eos_vector), 0)

        # Añade los tokens especiales al vocabulario
        vocab[sos_token] = len(vocab)
        vocab[eos_token] = len(vocab)

        words.append(sos_token)
        words.append(eos_token)

        vocabulary.itos = words
        vocabulary.stoi = vocab
        vocabulary.vectors = embedding_matrix
    
        return vocabulary
        

    def leer_europarl(self):
        with open(self.archivo_ingles, 'r', encoding='utf-8') as f_ingles, open(self.archivo_espanol, 'r', encoding='utf-8') as f_espanol:
            for oracion_ingles, oracion_espanol in zip(f_ingles, f_espanol):
                yield oracion_ingles.strip().lower(), oracion_espanol.strip().lower()

    def __len__(self):
        return len(self.ingles)

    def __getitem__(self, idx):
        if isinstance(idx, slice):
            items = [(self.ingles[idx], self.espanol[idx]) for idx in range(*idx.indices(len(self.ingles)))]
            items = [(self.vocab_en.get_vecs_by_tokens(self.tokenizer_en(item[0])), self.vocab_es.get_vecs_by_tokens(self.tokenizer_es(item[1]))) for item in items]

            if not items:
                raise RuntimeError("Todas las muestras en este lote están vacías.")
            
            # Añade el token de fin de secuencia a la oración en inglés
            # Añade el token de inicio y final de secuencia a la oración en español
            for item in items:
                if not item[0] or not item[1]:
                    raise RuntimeError("Una de las muestras en este lote está vacía.")
                eos_token = self.vocab_en.vectors[self.vocab_en.stoi['<eos>']].unsqueeze(0)
                sos_token = self.vocab_es.vectors[self.vocab_es.stoi['<sos>']].unsqueeze(0)
                eos_token = self.vocab_es.vectors[self.vocab_es.stoi['<eos>']].unsqueeze(0)
                item[0] = torch.cat((item[0], eos_token), 0)
                item[1] = torch.cat((sos_token, item[1], eos_token), 0)                
        
            # ing_tensors, esp_tensors = zip(*items)
            # ing_tensors = pad_sequence(ing_tensors, batch_first=True)
            # esp_tensors = pad_sequence(esp_tensors, batch_first=True)
            return items
        else:
            item = self.ingles[idx], self.espanol[idx]
            tokens_ingles = self.tokenizer_en(item[0])
            tokens_espanol = self.tokenizer_es(item[1])

            if not tokens_ingles or not tokens_espanol:
                return torch.zeros(1, 300), torch.zeros(1, 300)
                # raise RuntimeError("Una de las muestras está vacía.")
        
            tensor_ingles = self.vocab_en.get_vecs_by_tokens(tokens_ingles)
            tensor_espanol = self.vocab_es.get_vecs_by_tokens(tokens_espanol)

            # Añade el token de fin de secuencia a la oración en inglés
            eos_token = self.vocab_en.vectors[self.vocab_en.stoi['<eos>']].unsqueeze(0)
            # Añade el token de inicio y final de secuencia a la oración en español
            sos_token = self.vocab_es.vectors[self.vocab_es.stoi['<sos>']].unsqueeze(0)
            eos_token = self.vocab_es.vectors[self.vocab_es.stoi['<eos>']].unsqueeze(0)

            tensor_ingles = torch.cat((tensor_ingles, eos_token), 0)
            tensor_espanol = torch.cat((sos_token, tensor_espanol, eos_token), 0)

            return tensor_ingles, tensor_espanol
            
        
def collate_fn(batch):
    ingles_batch, espanol_batch = zip(*batch)
    ingles_batch = pad_sequence(ingles_batch, batch_first=True, padding_value=0)
    espanol_batch = pad_sequence(espanol_batch, batch_first=True, padding_value=0)
    return ingles_batch, espanol_batch

In [27]:
europarl_dataset = EuroparlDataset()

#### **Modelo**

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim

##### **Encoder**

In [29]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__() 
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, x):
        output, (hidden, cell) = self.rnn(x)
        return output, (hidden, cell)

##### **Decoder**

In [30]:
class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)  # TO DO: Añadir dropout 
        self.fc_out = nn.Linear(hidden_dim, input_dim)

    def forward(self, x, hidden, cell):
        output, (hidden, cell) = self.rnn(x, (hidden, cell))
        output = self.fc_out(output)
        return output, (hidden, cell)

In [31]:
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        target_len = target.shape[1]
        batch_size = target.shape[0]
        input_dim = target.shape[2]

        # Tensor para almacenar las salidas del decoder
        outputs = torch.zeros(batch_size, target_len, input_dim)
        
        # Primero, la fuente es procesada por el encoder
        _, (hidden, cell) = self.encoder(source)

        # La primera entrada al decoder es el vector <sos>
        x = target[:, 0, :]

        for t in range(1, target_len):
            output, (hidden, cell) = self.decoder(x.unsqueeze(1), hidden, cell)
            outputs[:, t, :] = output.squeeze(1)
            if random.random() < teacher_forcing_ratio:
                x = target[:, t, :]
            else:
                x = output.squeeze(1)

        return outputs

In [ ]:
# Parámetros
input_dim = 300
hidden_dim = 512
num_layers = 2
learning_rate = 0.001
num_epochs = 10
batch_size = 32

# Inicializa el modelo, el optimizador y la función de pérdida
encoder = Encoder(input_dim, hidden_dim, num_layers)
decoder = Decoder(input_dim, hidden_dim, num_layers)
model = Seq2Seq(encoder, decoder)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()  # Asumiendo que estás haciendo regresión. Cambia si es necesario.

# DataLoader
from torch.utils.data import DataLoader
dataloader = DataLoader(europarl_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Bucle de entrenamiento
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (src, tgt) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(src, tgt)
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / len(dataloader):.4f}')


# ----------------

In [35]:
from torch.utils.data import DataLoader

# Parámetros para el DataLoader
batch_size = 32  # Número de muestras por lote
shuffle = False   # Si se desea mezclar el conjunto de datos antes de crear lotes
num_workers = 0  # Número de subprocesos para cargar los datos

# Creación del DataLoader
data_loader = DataLoader(europarl_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate_fn)

batch = next(iter(data_loader))

r = model(batch[0], batch[1])

print(r.shape)


torch.Size([32, 100, 300])


In [ ]:

print(batch[0].shape)

out, (h, c) = enc(batch[0])

# input_hidden = torch.zeros(2, 2, 512)
i#nput_cell = torch.zeros(2, 2, 512)

input_x = torch.zeros(32, 111, 512)

out, (h_dec, c_dec) = dec(input_x, h, c)

print(out.shape)

In [63]:
input = torch.randn(32, 103, 300)

out, (hidden, cell) = enc(input)

print("hidden.shape enc:", hidden.shape)

x = torch.randn(32, 117, 512)

output, (hidden, cell) = dec(x, hidden, cell)

print("Output del dec:", output.shape)

hidden.shape enc: torch.Size([2, 32, 512])
Output del dec: torch.Size([32, 117, 512])


In [5]:
from torch.utils.data import DataLoader

# Parámetros para el DataLoader
batch_size = 32  # Número de muestras por lote
shuffle = False   # Si se desea mezclar el conjunto de datos antes de crear lotes
num_workers = 0  # Número de subprocesos para cargar los datos

# Creación del DataLoader
data_loader = DataLoader(europarl_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate_fn)

# DataLoader en el bucle de entrenamiento
for i, batch in enumerate(data_loader):
    ingles_batch, espanol_batch = batch
    # Aquí iría el código para procesar cada lote
    print('Inglés:', ingles_batch.shape)
    print('Español:', espanol_batch.shape)
    if i == 0:
        break  


Inglés: torch.Size([32, 103, 300])
Español: torch.Size([32, 98, 300])


In [6]:
print(ingles_batch[0,:,299])

tensor([-0.0111, -0.0397,  0.0354, -0.1321,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

In [7]:
!ls

data                nb01.ipynb          nb03-resuelto.ipynb nb04.ipynb
imgs                nb02.ipynb          nb03.ipynb          nb05.ipynb


In [54]:
a,(b,c) = 1,(2,3)

print(a)
print(b)
print(c)

1
2
3


1